In [12]:
import pandas as pd
import os

## Merge CSVs (or load if existing)

In [20]:
df = pd.DataFrame()

if os.path.exists('./data/merged.csv'):
    df = pd.read_csv('./data/merged.csv')
else:    
    files = [file for file in os.listdir('./data')]

    for file in files:
        frame = pd.read_csv(f'./data/{file}')
        df = pd.concat([df, frame])
        
    df.to_csv('./data/merged.csv', index=False)

df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,222910,Apple Airpods Headphones,1,150,07/26/19 16:51,"389 South St, Atlanta, GA 30301"
1,222911,Flatscreen TV,1,300,07/05/19 08:55,"590 4th St, Seattle, WA 98101"
2,222912,AA Batteries (4-pack),1,3.84,07/29/19 12:41,"861 Hill St, Atlanta, GA 30301"
3,222913,AA Batteries (4-pack),1,3.84,07/28/19 10:15,"190 Ridge St, Atlanta, GA 30301"
4,222914,AAA Batteries (4-pack),5,2.99,07/31/19 02:13,"824 Forest St, Seattle, WA 98101"


## Clean data

### Drop NaN

In [23]:
df = df.dropna(how='all')

## Augment data

### Add month column

In [24]:
df['Month'] = df['Order Date'].str[:2].astype('int32')
df.head()

ValueError: invalid literal for int() with base 10: 'Or'